This notebook contains code that was utilized for processing the demultiplexed amplicon sequences. Further processing may have been performed in mothur or R.

This chunk is for generating a dataframe for the real sequencing data (subsample 100 events) as well as an in silico data set based off of the Poisson distribution (accounting for 16S copy number). The dataframe is outputted into a shared file to be normalized by mothur (same sequencing depth across samples) and generate a principle component analysis chart (PCA). PCA was chosen because the number of dimensions is relatively small, and PCA tends to reduce the variability into 3 dimensions which captures the data better than using a dissimilarity matrix and using a PCoA, which won't capture much of the variability. The resulting PCA is plotted with ggplot in R and then edited with Illustrator to make the figure in the manuscript.

In [ ]:
import pandas as pd
import numpy as np

def in_silico_mock(lambdas, label, n_instances, seed, multiplier):
# Generate Poisson-distributed data for each lambda
    np.random.seed(seed)
    
    lambda_df = pd.DataFrame()

    Bs_poisson = np.random.poisson(lambdas[0], n_instances)
    Bt_poisson = np.random.poisson(lambdas[1], n_instances)
    Ec_poisson = np.random.poisson(lambdas[2], n_instances)
    Pp_poisson = np.random.poisson(lambdas[3], n_instances)
    
    # Create a DataFrame for the current lambda
    # Multiply by 16S copy number to account for that
    #Multiplier is for subsampling, to increase the number of events from all samples for normalization downstream
    lambda_df['Bacillus_subtilis'] = Bs_poisson*10*multiplier
    lambda_df['Bacteroides_thetaiotaomicron'] = Bt_poisson*5*multiplier
    lambda_df['Escherichia_coli'] = Ec_poisson*7*multiplier
    lambda_df['Pseudomonas_putida'] = Pp_poisson*7*multiplier

    #lambda_df['Total'] = lambda_df[['Bacillus_subtilis','Bacteroides_thetaiotaomicron','Escherichia_coli','Pseudomonas_putida']].sum(axis=1)

    #lambda_df['Bacillus_subtilis'] = lambda_df['Bacillus_subtilis']/lambda_df['Total']
    #lambda_df['Bacteroides_thetaiotaomicron'] = lambda_df['Bacteroides_thetaiotaomicron']/lambda_df['Total']
    #lambda_df['Escherichia_coli'] = lambda_df['Escherichia_coli']/lambda_df['Total']
    #lambda_df['Pseudomonas_putida'] = lambda_df['Pseudomonas_putida']/lambda_df['Total']

    lambda_df['label'] = 'vsearch'
    lambda_df['numOtus'] = 4
    lambda_df['Group'] = "silico_%s" % (label)

    #for row_index in lambda_df.index:
    #    lambda_df.iat[row_index,7] = row_index+1

    #lambda_df = lambda_df.drop('Total', axis='columns')

    shared_file_order = ['label', 'Group', 'numOtus', 'Bacillus_subtilis','Bacteroides_thetaiotaomicron','Escherichia_coli','Pseudomonas_putida' ]
    lambda_df = lambda_df[shared_file_order]

    return lambda_df

#generate in silico mock community
# Lambda values (in order of Bs, Bt, Ec, Pp)
lambdas_even = [25, 25, 25, 25]
lambdas_log1 = [0.8, 90, 0.2, 9]
lambdas_log2 = [9, 0.2, 90, 0.8]
# generate dummy data to force 4 sided PCA plot
#lambdas_log3 = [0.2, 0.8, 9, 90]
#lambdas_log4 = [90, 9, 0.8, 0.2]

# Initialize an empty DataFrame
silico_even_df = in_silico_mock(lambdas_even, "even", 100, 42, 3)
silico_log1_df = in_silico_mock(lambdas_log1, "log1", 100, 42, 3)
silico_log2_df = in_silico_mock(lambdas_log2, "log2", 100, 42, 3)
#silico_log3_df = in_silico_mock(lambdas_log3, "log3", 100, 42, 3)
#silico_log4_df = in_silico_mock(lambdas_log4, "log4", 100, 42, 3)

concat_silico_df = pd.concat([silico_even_df, silico_log1_df, silico_log2_df], axis = 0)
#concat_silico_df = pd.concat([silico_even_df, silico_log1_df, silico_log2_df, silico_log3_df, silico_log4_df], axis = 0)
#concat_silico_df.to_csv('silico_data.tsv', sep='\t', index=False)

#process the real mock community data
even_df = pd.read_csv('even/even.shared', delimiter = '\t')
log1_df = pd.read_csv('log1/log1.shared', delimiter = '\t')
log2_df = pd.read_csv('log2/log2.shared', delimiter = '\t')
                      
subsampled_even_df = even_df.sample(n=100, random_state=42)  # random_state for reproducibility
subsampled_log1_df = log1_df.sample(n=100, random_state=42)
subsampled_log2_df = log2_df.sample(n=100, random_state=42)

subsampled_even_df['Group'] = 'even'
subsampled_log1_df['Group'] = 'log1'
subsampled_log2_df['Group'] = 'log2'

concat_subsampled_df = pd.concat([subsampled_even_df, subsampled_log1_df, subsampled_log2_df], axis=0)

#Removed 16S copy number and multiplied that in the insilico data to account for it instead
concat_subsampled_df['Bacillus_subtilis'] = concat_subsampled_df['Bacillus_subtilis']
concat_subsampled_df['Bacteroides_thetaiotaomicron'] = concat_subsampled_df['Bacteroides_thetaiotaomicron']
concat_subsampled_df['Escherichia_coli'] = concat_subsampled_df['Escherichia_coli']
concat_subsampled_df['Pseudomonas_putida'] = concat_subsampled_df['Pseudomonas_putida']

#DO NOT NORMALIZE W/ REL ABUNDANCE, PCA WONT LIKE IT
#concat_subsampled_df['Total'] = concat_subsampled_df[['Bacillus_subtilis','Bacteroides_thetaiotaomicron','Escherichia_coli','Pseudomonas_putida']].sum(axis=1)
#concat_subsampled_df['Bacillus_subtilis'] = concat_subsampled_df['Bacillus_subtilis']/concat_subsampled_df['Total']
#concat_subsampled_df['Bacteroides_thetaiotaomicron'] = concat_subsampled_df['Bacteroides_thetaiotaomicron']/concat_subsampled_df['Total']
#concat_subsampled_df['Escherichia_coli'] = concat_subsampled_df['Escherichia_coli']/concat_subsampled_df['Total']
#concat_subsampled_df['Pseudomonas_putida'] = concat_subsampled_df['Pseudomonas_putida']/concat_subsampled_df['Total']
#concat_subsampled_df = concat_subsampled_df.drop('Total', axis='columns')
#concat_subsampled_df.to_csv('real_data.tsv', sep='\t', index=False)

#vertices of PCA space
ref_df = pd.DataFrame({
    'label': ['vsearch', 'vsearch', 'vsearch','vsearch'],
    'Group': ['Bs', 'Bt', 'Ec','Pp'],
    'numOtus': [4, 4, 4, 4],
    'Bacillus_subtilis': [1000, 0, 0, 0],
    'Bacteroides_thetaiotaomicron': [0, 1000, 0, 0],
    'Escherichia_coli': [0, 0, 1000, 0],
    'Pseudomonas_putida': [0, 0, 0, 1000]
})

#combine dataframes into one .shared file to be processed in mothur to subsample and to make the PCA plot
combined_data_df = pd.concat([concat_subsampled_df, concat_silico_df, ref_df], axis=0)
combined_data_df.to_csv('mock_pca_data.shared', sep='\t', index=False)
               

This code chunk converts the abundance dataframe into a presence/absence and then % prevelance and compares to expectation.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

def in_silico_mock(lambdas, label, n_instances, seed, multiplier):
# Generate Poisson-distributed data for each lambda
    np.random.seed(seed)
    
    lambda_df = pd.DataFrame()

    Bs_poisson = np.random.poisson(lambdas[0], n_instances)
    Bt_poisson = np.random.poisson(lambdas[1], n_instances)
    Ec_poisson = np.random.poisson(lambdas[2], n_instances)
    Pp_poisson = np.random.poisson(lambdas[3], n_instances)
    
    # Create a DataFrame for the current lambda
    # Multiply by 16S copy number to account for that
    #Multiplier is for subsampling, to increase the number of events from all samples for normalization downstream
    lambda_df['Bacillus_subtilis'] = Bs_poisson*10*multiplier
    lambda_df['Bacteroides_thetaiotaomicron'] = Bt_poisson*5*multiplier
    lambda_df['Escherichia_coli'] = Ec_poisson*7*multiplier
    lambda_df['Pseudomonas_putida'] = Pp_poisson*7*multiplier

    #lambda_df['Total'] = lambda_df[['Bacillus_subtilis','Bacteroides_thetaiotaomicron','Escherichia_coli','Pseudomonas_putida']].sum(axis=1)

    #lambda_df['Bacillus_subtilis'] = lambda_df['Bacillus_subtilis']/lambda_df['Total']
    #lambda_df['Bacteroides_thetaiotaomicron'] = lambda_df['Bacteroides_thetaiotaomicron']/lambda_df['Total']
    #lambda_df['Escherichia_coli'] = lambda_df['Escherichia_coli']/lambda_df['Total']
    #lambda_df['Pseudomonas_putida'] = lambda_df['Pseudomonas_putida']/lambda_df['Total']

    lambda_df['label'] = 'vsearch'
    lambda_df['numOtus'] = 4
    lambda_df['Group'] = "silico_%s" % (label)

    #for row_index in lambda_df.index:
    #    lambda_df.iat[row_index,7] = row_index+1

    #lambda_df = lambda_df.drop('Total', axis='columns')

    shared_file_order = ['label', 'Group', 'numOtus', 'Bacillus_subtilis','Bacteroides_thetaiotaomicron','Escherichia_coli','Pseudomonas_putida' ]
    lambda_df = lambda_df[shared_file_order]

    return lambda_df

def prevalence(df):
    df.loc[:,'Bacillus_subtilis':'Pseudomonas_putida']=np.where(df.loc[:,'Bacillus_subtilis':'Pseudomonas_putida']>0,1,0)
    tot_inst = df.shape[0]
    df_prev_sums = df.loc[:,'Bacillus_subtilis':'Pseudomonas_putida'].sum()*100/tot_inst
    return df_prev_sums

def average_rel_abun(df):
    df['Total'] = df.loc[:,'Bacillus_subtilis':'Pseudomonas_putida'].sum(axis=1)
    df['Bacillus_subtilis'] = df['Bacillus_subtilis']*100/df['Total']
    df['Escherichia_coli'] = df['Escherichia_coli']*100/df['Total']
    df['Bacteroides_thetaiotaomicron'] = df['Bacteroides_thetaiotaomicron']*100/df['Total']
    df['Pseudomonas_putida'] = df['Pseudomonas_putida']*100/df['Total']

    avg_rel_abun = []
    avg_rel_abun[0] = df[df['Bacillus_subtilis'] != 0]['Bacillus_subtilis'].mean()
    avg_rel_abun[1] = df[df['Bacteroides_thetaiotaomicron'] != 0]['Bacteroides_thetaiotaomicron'].mean()
    avg_rel_abun[2] = df[df['Escherichia_coli'] != 0]['Escherichia_coli'].mean()
    avg_rel_abun[3] = df[df['Pseudomonas_putida'] != 0]['Pseudomonas_putida'].mean()

    return

#generate in silico mock community
# Lambda values (in order of Bs, Bt, Ec, Pp)
lambdas_even = [25, 25, 25, 25]
lambdas_log1 = [0.8, 90, 0.2, 9]
lambdas_log2 = [9, 0.2, 90, 0.8]

# Initialize an empty DataFrame
silico_even_df = in_silico_mock(lambdas_even, "even", 1000, 42, 3)
silico_log1_df = in_silico_mock(lambdas_log1, "log1", 1000, 42, 3)
silico_log2_df = in_silico_mock(lambdas_log2, "log2", 1000, 42, 3)

silico_even_df_prev_sums = prevalence(silico_even_df)
silico_log1_df_prev_sums = prevalence(silico_log1_df)
silico_log2_df_prev_sums = prevalence(silico_log2_df)

prev_data  = {
    'library': ['even_real', 'even_silico', 'log-1_real', 'log-1_silico', 'log-2_real', 'log-2_silico'],
    'Bs': [0,0,0,0,0,0],
    'Bt': [0,0,0,0,0,0],
    'Ec': [0,0,0,0,0,0],
    'Pp': [0,0,0,0,0,0]
}

prev_rar100_df = pd.DataFrame(prev_data)
prev_rar100_df.iloc[1,1:5] = silico_even_df_prev_sums
prev_rar100_df.iloc[3,1:5] = silico_log1_df_prev_sums
prev_rar100_df.iloc[5,1:5] = silico_log2_df_prev_sums

#process the real mock community data
even_df = pd.read_csv('even/even.shared', delimiter = '\t')
log1_df = pd.read_csv('log1/log1.shared', delimiter = '\t')
log2_df = pd.read_csv('log2/log2.shared', delimiter = '\t')
                      
even_df_prev_sums_rar100 = prevalence(even_df)
log1_df_prev_sums_rar100 = prevalence(log1_df)
log2_df_prev_sums_rar100 = prevalence(log2_df)

prev_rar100_df.iloc[0,1:5] = even_df_prev_sums_rar100
prev_rar100_df.iloc[2,1:5] = log1_df_prev_sums_rar100
prev_rar100_df.iloc[4,1:5] = log2_df_prev_sums_rar100

df_long = prev_rar100_df.melt(id_vars='library', var_name='Species', value_name='Prevalence(%)')
print(df_long)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(3, 2))

sns.scatterplot(data=df_long[df_long['library']=='even_real'], x="Species", y="Prevalence(%)",marker='o',ax=axes[0])
sns.scatterplot(data=df_long[df_long['library']=='even_silico'], x="Species", y="Prevalence(%)",marker='X',ax=axes[0])
axes[0].set_title('Even')
axes[0].set_ylim(0, 110)

sns.scatterplot(data=df_long[df_long['library']=='log-1_real'], x="Species", y="Prevalence(%)",marker='o',ax=axes[1])
sns.scatterplot(data=df_long[df_long['library']=='log-1_silico'], x="Species", y="Prevalence(%)",marker='X',ax=axes[1])
axes[1].set_title('Bt-biased')
axes[1].set_ylim(0, 110)
axes[1].set(yticklabels=[])
axes[1].set(ylabel=None)

sns.scatterplot(data=df_long[df_long['library']=='log-2_real'], x="Species", y="Prevalence(%)",marker='o',ax=axes[2])
sns.scatterplot(data=df_long[df_long['library']=='log-2_silico'], x="Species", y="Prevalence(%)",marker='X',ax=axes[2])
axes[2].set_title('Ec-biased')
axes[2].set_ylim(0, 110)
axes[2].set(yticklabels=[])
axes[2].set(ylabel=None)

plt.tight_layout()
plt.savefig('mock_prevalence.pdf')

Going to nix this code chunk since I don't think it shows anything super informative and I just want to finish this manuscript

This code chunk processes the data in a similar process to the above (also generating Poisson "null model" for comparison) but keeps the "even", and "log" samples separate and rarefies the data by specified percentages to demonstrate the effect of sequencing depth on the metrics of richness and prevalence of the rarer members

In [ ]:
import pandas as pd
import numpy as np
import random

def in_silico_mock(lambdas, label, n_instances, seed, multiplier):
# Generate Poisson-distributed data for each lambda
    np.random.seed(seed)
    
    lambda_df = pd.DataFrame()

    Bs_poisson = np.random.poisson(lambdas[0], n_instances)
    Bt_poisson = np.random.poisson(lambdas[1], n_instances)
    Ec_poisson = np.random.poisson(lambdas[2], n_instances)
    Pp_poisson = np.random.poisson(lambdas[3], n_instances)
    
    # Create a DataFrame for the current lambda
    # Multiply by 16S copy number to account for that
    #Multiplier is for subsampling, to increase the number of events from all samples for normalization downstream
    lambda_df['Bacillus_subtilis'] = Bs_poisson*10*multiplier
    lambda_df['Bacteroides_thetaiotaomicron'] = Bt_poisson*5*multiplier
    lambda_df['Escherichia_coli'] = Ec_poisson*7*multiplier
    lambda_df['Pseudomonas_putida'] = Pp_poisson*7*multiplier

    lambda_df['label'] = 'vsearch'
    lambda_df['numOtus'] = 4
    lambda_df['Group'] = "silico_%s" % (label)

    shared_file_order = ['label', 'Group', 'numOtus', 'Bacillus_subtilis','Bacteroides_thetaiotaomicron','Escherichia_coli','Pseudomonas_putida']
    lambda_df = lambda_df[shared_file_order]

    return lambda_df

def rarefy(df, group_name, rarefy_frac, sequencing_depth_threshold):
    #Specify the columns with integer values
    int_columns = df.columns[3:7]
    abun_df = df[int_columns]
    total_events = abun_df.sum().sum()

    tuple_list = []

    #iterate through the df and generate the list of locations
    for index, row in abun_df.iterrows():
        for col, value in row.items():
            tuple_list.extend([(index,col)] * value)

    #subsample
    subsamp_list = random.sample(tuple_list, int(rarefy_frac * total_events))

    #convert subsampled list back into a dataframe
    rarefy_df = df.copy()
    rarefy_df['label'] = 'vsearch'
    rarefy_df['numOtus'] = 4
    rarefy_df['Group'] = group_name
    rarefy_df['Bacillus_subtilis'] = 0
    rarefy_df['Bacteroides_thetaiotaomicron'] = 0
    rarefy_df['Escherichia_coli'] = 0
    rarefy_df['Pseudomonas_putida'] = 0

    #populate rarefied df based off of tuple list
    for entry in subsamp_list:
        row_index, col_name = entry
        rarefy_df.at[row_index,col_name] = rarefy_df.at[row_index,col_name] + 1

    #remove empty samples and samples below the sequencing depth threshold
    rarefy_df['Total'] = rarefy_df[['Bacillus_subtilis','Bacteroides_thetaiotaomicron','Escherichia_coli','Pseudomonas_putida']].sum(axis=1)
    rarefy_df = rarefy_df[rarefy_df['Total'] != 0]
    rarefy_df = rarefy_df[rarefy_df['Total'] >= sequencing_depth_threshold]
    rarefy_df = rarefy_df.drop('Total', axis=1)
    
    return rarefy_df

#process the real mock community data
even_df = pd.read_csv('even/even.shared', delimiter = '\t')
log1_df = pd.read_csv('log1/log1.shared', delimiter = '\t')
log2_df = pd.read_csv('log2/log2.shared', delimiter = '\t')

#determine number of samples per sample
num_even = len(even_df.index)
num_log1 = len(log1_df.index)
num_log2 = len(log2_df.index)
                      
#generate in silico mock community
# Lambda values (in order of Bs, Bt, Ec, Pp)
lambdas_even = [25, 25, 25, 25]
lambdas_log1 = [0.8, 90, 0.2, 9]
lambdas_log2 = [9, 0.2, 90, 0.8]

silico_even_df = in_silico_mock(lambdas_even, "even", num_even, 42, 3)
silico_log1_df = in_silico_mock(lambdas_log1, "log1", num_log1, 42, 3)
silico_log2_df = in_silico_mock(lambdas_log2, "log2", num_log2, 42, 3)

even_df['Group'] = 'even'
log1_df['Group'] = 'log1'
log2_df['Group'] = 'log2'

#For testing
# data = {'label': [0.03, 0.03, 0.03, 0.03],
#         'Group': ['test','test','test','test'],
#         'numOtus':[4,4,4,4],
#         'Bacillus_subtilis':[0,40,90,0],
#         'Bacteroides_thetaiotaomicron':[50,10,10,30],
#         'Escherichia_coli':[0,50,0,70],
#         'Pseudomonas_putida':[50,0,0,0]}
# test_df = pd.DataFrame(data)

log1_rarefy_10_df = rarefy(log1_df, 'log1_10', 0.1, 100)
#log1_rarefy_25_df = rarefy(log1_df, 'log1_25', 0.25, 100)
#log1_rarefy_50_df = rarefy(log1_df, 'log1_50', 0.50, 100)
#log1_rarefy_100_df = rarefy(log1_df, 'log1_100', 100)

print(log1_rarefy_10_df)



        